In [3]:
#imports 
from statsbombpy import sb
import mplsoccer

import pandas as pd
import numpy as np
import math

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from copy import deepcopy
from ast import literal_eval

In [4]:
#df settings
pd.set_option("display.max_rows", 150)
pd.set_option("display.max_columns", 50)

In [6]:
data = pd.read_csv('played.csv')

In [7]:

def enter_team():
    player_names =literal_eval(input('Enter your team and their GW opponents as a dictionary'))
    team = pd.DataFrame()
    team['player_name']=list(player_names.keys())
    team['opponent_team'] = list(player_names.values())
    teams = np.array([])
    #print(team)
    for name in list(team['player_name']):
        i=0
        try:
            player_data = data[data['name']==name]
            while not player_data['team_x'].iloc[i]:
                i+=1
                continue
            teams = np.append(teams , player_data['team_x'].iloc[i])
        except:
            teams=np.append(teams,np.nan)
    team['latest_team'] = teams
    print(team)
    return team


    

In [ ]:
exp_points=np.array([])
for i in range(len(team['player_name'])):
    filter_data = data[(data['name']==team['player_name'].iloc[i]) & (data['opp_team_name']==team['opponent_team'].iloc[i])]['total_points']
    #print(filter_data.mean())
    exp_points=np.append(exp_points, filter_data.mean())
team['expected_points']=exp_points    
print(exp_points)

In [9]:
def get_fixtures(fixture_string,change_names_dict):
    fixture_list = fixture_string.split(sep=' , ')
    #print(fixture_list)
    fixture_dict = {}
    for i in fixture_list:
        l1 = i.split(sep=' v ')
        #print(l1)
        if change_names_dict.get(l1[0])!=None:
            l1[0]=change_names_dict[l1[0]]
        if change_names_dict.get(l1[1])!=None:
            l1[1]=change_names_dict[l1[1]]    
        fixture_dict[l1[0]]=l1[1]
        fixture_dict[l1[1]]=l1[0]
        
    return fixture_dict

In [10]:
string1 = 'Aston Villa v Crystal Palace , Arsenal v Bournemouth , Brighton v West Ham , Manchester City v Newcastle , Chelsea v Leeds , Wolves v Spurs , Brentford v Fulham , Nottm Forest v Everton , Liverpool v Man Utd , Leicester v Southampton'

In [11]:
change_names_dict = {'Leicester City': 'Leicester','Newcastle United':'Newcastle','Manchester City':'Man City','AFC Bournemouth':'Bournemouth','Leeds United':'Leeds','Manchester United':'Man Utd','Tottenham Hotspur':'Spurs','West Ham United':'West Ham'}

In [22]:
fixtures = get_fixtures(string1,change_names_dict)
recc_list=pd.DataFrame()
for team in list(fixtures.keys()):
    data_subset = data[(data['opp_team_name']==team) & (data['team_x']==fixtures[team])]
    if len(data_subset)==0:
        continue
    data_subset = data_subset.groupby(['name','opp_team_name'])['total_points'].agg(['mean','count'])
    data_subset.sort_values(by='mean',ascending=False,inplace=True)
    #print(data_subset.head())
    recc_list = recc_list.append(data_subset.head(4))

In [23]:
recc_list

,,mean,count
name,opp_team_name,,
Marc Guéhi,Aston Villa,9.000000,1
Eberechi Eze,Aston Villa,4.000000,3
Tyrick Mitchell,Aston Villa,3.500000,4
Vicente Guaita,Aston Villa,2.600000,5
Ollie Watkins,Crystal Palace,6.000000,3
John McGinn,Crystal Palace,4.800000,5
Anwar El Ghazi,Crystal Palace,4.200000,5
Ashley Young,Crystal Palace,3.333333,9
Pierre-Emerick Aubameyang,Bournemouth,6.000000,4


In [24]:
#removing entries with less than two apprearences agianst the opposition
recc_list.drop(recc_list[(recc_list['count']<2) | (recc_list['mean']<=3)].index,inplace=True)

In [25]:
recc_list.reset_index(inplace=True)

In [26]:
recc_list


,name,opp_team_name,mean,count
0,Eberechi Eze,Aston Villa,4.000000,3
1,Tyrick Mitchell,Aston Villa,3.500000,4
2,Ollie Watkins,Crystal Palace,6.000000,3
3,John McGinn,Crystal Palace,4.800000,5
4,Anwar El Ghazi,Crystal Palace,4.200000,5
5,Ashley Young,Crystal Palace,3.333333,9
6,Pierre-Emerick Aubameyang,Bournemouth,6.000000,4
7,Sead Kolasinac,Bournemouth,5.500000,6
8,David Luiz Moreira Marinho,Bournemouth,4.875000,8
9,Bernd Leno,Bournemouth,3.750000,4


In [29]:
#dropping irrelevant players
recc_list.drop([5,6,7,8,9,25,19,21,22,26,32],inplace=True)

In [32]:
recc_list.columns

Index(['name', 'opp_team_name', 'mean', 'count'], dtype='object')

In [33]:
recc_list

,name,opp_team_name,mean,count
0,Eberechi Eze,Aston Villa,4.000000,3
1,Tyrick Mitchell,Aston Villa,3.500000,4
2,Ollie Watkins,Crystal Palace,6.000000,3
3,John McGinn,Crystal Palace,4.800000,5
4,Anwar El Ghazi,Crystal Palace,4.200000,5
10,Tomas Soucek,Brighton,4.800000,5
11,Pascal Groß,West Ham,5.200000,10
12,Danny Welbeck,West Ham,3.166667,12
13,Lewis Dunk,West Ham,3.100000,10
14,João Pedro Cavaco Cancelo,Newcastle,7.666667,6


In [34]:
recc_list.drop([15,16],inplace=True)

In [35]:
recc_list.reset_index(inplace=True)

In [36]:
recc_list

,index,name,opp_team_name,mean,count
0,0,Eberechi Eze,Aston Villa,4.000000,3
1,1,Tyrick Mitchell,Aston Villa,3.500000,4
2,2,Ollie Watkins,Crystal Palace,6.000000,3
3,3,John McGinn,Crystal Palace,4.800000,5
4,4,Anwar El Ghazi,Crystal Palace,4.200000,5
5,10,Tomas Soucek,Brighton,4.800000,5
6,11,Pascal Groß,West Ham,5.200000,10
7,12,Danny Welbeck,West Ham,3.166667,12
8,13,Lewis Dunk,West Ham,3.100000,10
9,14,João Pedro Cavaco Cancelo,Newcastle,7.666667,6


In [37]:
recc_list[['name', 'opp_team_name', 'mean', 'count']].to_

,name,opp_team_name,mean,count
0,Eberechi Eze,Aston Villa,4.000000,3
1,Tyrick Mitchell,Aston Villa,3.500000,4
2,Ollie Watkins,Crystal Palace,6.000000,3
3,John McGinn,Crystal Palace,4.800000,5
4,Anwar El Ghazi,Crystal Palace,4.200000,5
5,Tomas Soucek,Brighton,4.800000,5
6,Pascal Groß,West Ham,5.200000,10
7,Danny Welbeck,West Ham,3.166667,12
8,Lewis Dunk,West Ham,3.100000,10
9,João Pedro Cavaco Cancelo,Newcastle,7.666667,6
